In [14]:
import numpy as np
import pandas as pd
from pandas import Series
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8')
sns.set(font_scale=2.5)

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [15]:
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')
df_train['FamliySize'] = df_train['SibSp'] + df_train['Parch'] + 1
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] + 1

df_test.loc[df_test.Fare.isnull(), 'Fare'] = df_test['Fare'].mean()

df_train['Fare'] = df_train['Fare'].map(lambda i : np.log(i) if i>0 else 0)
df_test['Fare'] = df_test['Fare'].map(lambda i: np.log(i) if i>0 else 0)

3. Feature engineering

    - 본격적인 feature engineering을 시작해보겠습니다.
    - 가장 먼저, dataset에 존재하는 null data를 채우려고함
    - 아무 숫자로 채울 수는 없고, null data를 포함하는 fearue의 statics를 참고하거나, 다른 아이디어를 짜내어 채울 수 있습니다.
    - null data를 어떻게 채우느냐에 따라 모델의 성능이 좌지우지 될 수 있기 때문에, 신경써줘야할 부분입니다.
    - Feature engineering은 실제 모델의 학습에 쓰려고 하는 것이므로, train뿐만 아니라, test도 똑같이 적용해주어야 합니다.  
    잊지마세요

3.1 Fill Null

    3.1.1 Fill null in Age using title
        - Age에는 null data가 177개나 있습니다. 이를 채울 수 있는 여러 아이디어가 있을 건인데, 여기서 우리는 title + statistic을 사용해보도록 하겠습니다.
        - 영어에서는 miss, mr, mrs 와 같은 title이 존재합니다. 각 탑승객의 이름에는 꼭 이런 title이 들어가게되는 이를 사용해보도록 하겠습니다.
        - pandas series에는 data를 string으로 바꾸주는 str method, 거기에 정규표현식을 적용하게 해주는  extract method가 있습니다.
        이를 사용하여 title을 쉽게 추출할 수 잇습니다.
        - title을 initial column에 저장하겠습니다.

In [16]:
df_train['Initial'] = df_train.Name.str.extract('([A-Za-z]+)\.')    # Lets extract the solutations
df_test['Initial']=df_test.Name.str.extract('([A-Za-z]+)\.')        # Lets extract the solutations

        -pandas의 corsstab을 이용하여 우리가 추출한 Initial과 Sex간의 count를 살펴봅시다.

In [17]:
# Checking the Initials with the Sex
pd.crosstab(df_train['Initial'], df_train['Sex']).T.style.background_gradient(cmap='summer_r')

Initial,Capt,Col,Countess,Don,Dr,Jonkheer,Lady,Major,Master,Miss,Mlle,Mme,Mr,Mrs,Ms,Rev,Sir
Sex,,,,,,,,,,,,,,,,,
female,0,0,1,0,1,0,1,0,0,182,2,1,0,125,1,0,0
male,1,2,0,1,6,1,0,2,40,0,0,0,517,0,0,6,1


        - 위 테이블을 참고하여 남자, 여자가 쓰는 initial을 구분해보곗습니다.
        replace 메소드를 사용하면 특정 데이터 값을 원하는 값으로 치환해줍니다

In [18]:
df_train['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],
                        ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mr'],inplace=True)

df_test['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],
                        ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mr'],inplace=True)

df_train.groupby('Initial').mean()


TypeError: Could not convert Palsson, Master. Gosta LeonardRice, Master. EugenePanula, Master. Juha NiiloGoodwin, Master. William FrederickSkoog, Master. HaraldMoubarek, Master. GeriosCaldwell, Master. Alden GatesNicola-Yarred, Master. EliasSage, Master. Thomas HenryPanula, Master. Eino ViljamiGoldsmith, Master. Frank John William "Frankie"Rice, Master. ArthurLefebre, Master. Henry ForbesAsplund, Master. Clarence Gustaf HugoBecker, Master. Richard FNavratil, Master. Michel MAsplund, Master. Edvin Rojj FelixRice, Master. EricAllison, Master. Hudson TrevorNavratil, Master. Edmond RogerCoutts, Master. William Loch "William"Goodwin, Master. Sidney LeonardRichards, Master. William RoweDodge, Master. WashingtonGoodwin, Master. Harold VictorCoutts, Master. Eden Leslie "Neville"Davies, Master. John Morgan JrMoubarek, Master. Halim Gonios ("William George")Moor, Master. MeierHamalainen, Master. ViljoRice, Master. George HughDean, Master. Bertram VereCarter, Master. William Thornton IIThomas, Ma